In [120]:
from datetime import datetime, timedelta

DATE_FORMAT = '%Y-%m-%d'
DOW_NUM = {
    0:'Sunday',
    1:'Monday',
    2:'Tuesday',
    3:'Wednesday',
    4:'Thursday',
    5:'Friday',
    6:'Saturday'
}

## Easy method

In [2]:
def dow_datetime_method(date):
    dow_name = datetime.strptime(date, DATE_FORMAT).strftime('%A')
    return dow_name

dow_datetime_method('1995-01-24')

'Tuesday'

## Using the doomsday series

In [106]:
def _mod(n, modulo):
    """
    "absolute" modulo
    """
    if n < 0:
        shift = (n)%-modulo
    elif n > 0:
        shift = (n)%modulo
    else:
        shift = 0
    return shift

def _century_leaps_adj(ref_year, target_year):
    """
    Accounting for leap years not happening on years divisible by 100 but not 400.
    """
    if ref_year > target_year:
        century_list = [i for i in range(target_year+1,ref_year+1) if i%100==0 and i%400!=0]
        adj = len(century_list)
    elif ref_year < target_year:
        century_list = [i for i in range(ref_year,target_year+1) if i%100==0 and i%400!=0]
        adj = -len(century_list)
    else:
        adj = 0
    return adj

def dow_doomsday_complex(date):

    ref_year={2000:2}
    year = int(datetime.strptime(date, DATE_FORMAT).strftime('%Y'))

    year_diff = year - list(ref_year.keys())[0]

    if year_diff < 0:
        leap_year_count = -(year_diff//-4)-1
        if year_diff%-4 == 0:
            leap_year_count = leap_year_count + 1
    elif year_diff > 0:
        leap_year_count = year_diff//4
    else:
        leap_year_count = 0

    century_leaps = _century_leaps_adj(ref_year=list(ref_year.keys())[0], target_year=year)

    year_shift = leap_year_count + year_diff + century_leaps

    doomsday = datetime(year,6,6)

    day_shift = (datetime.strptime(date, DATE_FORMAT) - doomsday).days

    total_shift = _mod(year_shift+day_shift, 7)
    final_dow = list(ref_year.values())[0] + total_shift

    if final_dow < 0:
        final_dow = final_dow + 7
    elif final_dow > 6:
        final_dow = final_dow - 7

    return DOW_NUM[final_dow]

date2 = '2000-06-06'
print(dow_datetime_method(date2))
print(dow_doomsday_complex(date2))

Tuesday
Tuesday


In [182]:
def _get_doomsday_list(year):
    doomsdays = [
        '04-04',
        '06-06',
        '08-08',
        '10-10',
        '12-12',
        '07-11',
        '11-07',
        '05-09',
        '09-05'
    ]
    leap_year_doomsdays = [
        '01-04',
        '02-29'
    ]
    non_leap_year_doomsdays = [
        '01-03',
        '02-28'
    ]
    if year%4!=0 or (year%100==0 and year%400!=0):
        all_doomsdays = non_leap_year_doomsdays + doomsdays
    else:
        all_doomsdays = leap_year_doomsdays + doomsdays
    l = []

    string_year = str(year)
    while len(string_year)<4:
        string_year = f'0{string_year}'

    for i in sorted(all_doomsdays):
        l = l + [f'{string_year}-{i}']

    return l

def _get_century_start_doomsday(year):
    century_dow_seq = {
        0:2,
        1:6,
        2:5,
        3:3
    }

    return century_dow_seq[(year//100)%4]

def dow_doomsday_human(date):
    year = int(datetime.strptime(date, DATE_FORMAT).strftime('%Y'))

    century_start_doomsday = _get_century_start_doomsday(year)

    a = (year%100)//12
    b = (year%100)%12
    year_adjust = a + b + b//4

    doomsday_list = _get_doomsday_list(year)

    if datetime.strptime(doomsday_list[0], DATE_FORMAT) > datetime.strptime(date, DATE_FORMAT):
        closest_greater_doomsday = datetime.strptime(doomsday_list[0], DATE_FORMAT)
        day_adj = (datetime.strptime(date, DATE_FORMAT) - closest_greater_doomsday).days
    else:
        doomsday_list
        closest_smaller_doomsday = datetime.strptime([i for i in doomsday_list if i <= date][-1], DATE_FORMAT)
        day_adj = (datetime.strptime(date, DATE_FORMAT) - closest_smaller_doomsday).days

    final_dow = (century_start_doomsday + year_adjust + day_adj)%7

    if final_dow < 0:
        final_dow = final_dow + 7
    elif final_dow > 6:
        final_dow = final_dow - 7
    return DOW_NUM[final_dow]

date = '0001-01-03'
print(dow_datetime_method(date))
print(dow_doomsday_human(date))


Wednesday
Wednesday


In [184]:
dates = ['0001-01-03','1599-02-28','1904-02-29','2022-01-31','9999-12-31']

[dow_datetime_method(i) for i in dates]

['Wednesday', 'Sunday', 'Monday', 'Monday', 'Friday']